In [1]:
import pickle
import numpy as np
import os
import sys
import json
import torch

In [3]:
import pickle

with open("data.pkl", "rb") as f:
    reloaded_data = pickle.load(f)
    
reloaded_data[:3]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 \nEdition \nUNIVERSITY OF HAWAI‘I AT MĀNOA \nFOOD SCIENCE AND HUMAN \nNUTRITION PROGRAM \nALAN TITCHENAL, SKYLAR HARA, \nNOEMI ARCEO CAACBAY, WILLIAM \nMEINKE-LAU, YA-YUN YANG, MARIE \nKAINOA FIALKOWSKI REVILLA, \nJENNIFER DRAPER, GEMADY \nLANGFELDER, CHERYL GIBBY, CHYNA \nNICOLE CHUN, AND ALLISON \nCALABRESE',
  'chunk_char_count': 320,
  'chunk_word_count': 42,
  'chunk_token_count': 80.0,
  'id': '1bb0eb1b87a146f8986f42cfe65bcc4f',
  'embedding_numpy': array([-0.01989893,  0.01859011,  0.0209169 , ...,  0.00927082,
         -0.01116134, -0.01302762])},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and \nHuman Nutrition Program is licensed under a Creative Commons Attribution 4.0 \nInternational License, except where otherwise noted.',
  'chunk_char_count': 212,
  'chunk_word_count': 30,
  'chunk_token_count': 53.0,
  'id': 'f4c9c0382f4c450fbf051e

In [4]:
embedding_np_list = [content['embedding_numpy'] for content in reloaded_data] 

In [5]:
embed_tensor = torch.tensor(np.array(embedding_np_list), dtype=torch.float32)

In [5]:
embed_tensor.shape

torch.Size([1683, 1536])

In [6]:
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

In [21]:
! pip install openai transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 123.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 110.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 42.6 MB/s eta 0:00:00


In [7]:
from openai import OpenAI

openai_client = OpenAI(api_key=OPEN_API_KEY)

def get_text_query(query, model="text-embedding-3-small"):
    query = str(query).replace("\n", " ")
    embedd_query = openai_client.embeddings.create(input = [query], model=model).data[0].embedding
    embedd_query_tensor = torch.tensor(np.array(embedd_query), dtype=torch.float32)
    sim_tensor = util.dot_score(a=embedd_query_tensor, b=embed_tensor)
    top_results = torch.topk(sim_tensor[0], k=5)
    return top_results.values, top_results.indices
    

In [23]:
from sentence_transformers import util

sim_tensor = util.dot_score(a=get_text_query("How does nutrition improve"), b=embed_tensor)

In [24]:
top_results = torch.topk(sim_tensor[0], k=5)

In [25]:
top_results

torch.return_types.topk(
values=tensor([0.5442, 0.5386, 0.5367, 0.5326, 0.5239]),
indices=tensor([1657, 1323,   62, 1610, 1322]))

In [28]:
for index in top_results.indices:
    print(reloaded_data[index]['sentence_chunk'])

Working toward Tomorrow 
Whether you pursue nutrition as a career or simply work to improve 
your own dietary choices, what you have learned in this course can 
provide a solid foundation for the future. Remember, your ability to 
wake up, to think clearly, communicate, hope, dream, go to school, 
gain knowledge, and earn a living are totally dependent upon one 
factor—your health. Good health allows you to function normally 
and work hard to pursue your goals. Yet, achieving optimal health 
cannot be underestimated. It is a complex process, involving 
multiple dimensions of wellness, along with your physical or 
medical reality. The knowledge you have now acquired is also key.
However, it is not enough to pass this nutrition class with good 
grades. Nutrition knowledge must be applied to make a difference 
in your life, throughout your life.
Throughout this textbook, we have focused on the different 
aspects of nutrition science, which helps to optimize health and 
prevent disease. Sc

In [29]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# hf_token = "hf_XXXXX-token-here-XXXX

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it", token=hf_token)
# model = AutoModelForCausalLM.from_pretrained(
#     "google/gemma-7b-it",
#     torch_dtype=torch.bfloat16,
#     token = hf_token
# )

# input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt")

# outputs = model.generate(**input_ids)
# print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In the realm of data, a tale unfolds,


In [30]:
model.to("cuda:0")

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): Gemm

In [31]:
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda:0")

In [32]:
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In the realm of data, a tale unfolds,


In [13]:
from sentence_transformers import util

In [14]:
query = "Provide which way can vitamin C be increased?"

_, top_results_index = get_text_query(query)

top_results_index

tensor([ 807,  811, 1268,  812,  805])

In [15]:
contents_list = [reloaded_data[index]['sentence_chunk'] for index in top_results_index]

In [16]:
chat_prompt2 = f"""
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
<context1>{contents_list[0]}</context1>
<context2>{contents_list[1]}</context2>
<context3>{contents_list[2]}</context3>
\nRelevant passages: <extract relevant passages from the context here>
User query: <query>{query}</query>
Answer:
"""

In [ ]:
chat_content = f"""
based on the following context provided below answer the question
<content1>{contents_list[0]}</content1>
<content2>{contents_list[1]}</content2>
<content3>{contents_list[2]}</content3>

Query: {query}
"""

In [64]:

chat = [
    { "role": "user", "content": chat_prompt2},
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

In [65]:
prompt

"<bos><start_of_turn>user\nBased on the following context items, please answer the query.\nGive yourself room to think by extracting relevant passages from the context before answering the query.\nDon't return the thinking, only return the answer.\nMake sure your answers are as explanatory as possible.\nUse the following examples as reference for the ideal answer style.\nExample 1:\nQuery: What are the fat-soluble vitamins?\nAnswer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.\n\nExample 2:\nQuery: What are the causes of type 2 diabetes?\nAnswer: Type 2 diabetes is

In [66]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.

Example 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associ

In [70]:
torch.cuda.empty_cache()

In [11]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 47.2 MB/s eta 0:00:0000:0100:01


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", quantization_config=quantization_config)


`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:

chat = [
    { "role": "user", "content": chat_prompt2},
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

In [18]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.

Example 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associ